In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cProfile, pstats, sys
import cv2
import sys
import csv
import pickle

from modules.path_planning.astar import AStar
from modules.path_planning.BFS import BFS
from modules.path_planning.greedyBestFirst import greedyBestFirst
from modules.path_planning.bidirectDijkstras import BiDirectionalDijkstra
from modules.path_planning.RRT_star import RRT_Star
from modules.path_planning.INFORMED_RRT import Informed_RRT_star
from modules.path_planning.RRT import RRTtree
from modules.path_planning.RRA import RRA
from modules.path_planning.BIT import BIT, ELLIPSE
from modules.path_planning.Theta import Theta
from modules.path_planning.FIXED_ARA import ARA
from modules.path_planning.DLITE import DLITESEARCH
from modules.path_planning.ANA import ANA
from modules.path_planning.FIXED_ADA import MODIFIED_ADA
#from modules.path_planning.NEWBIT import DYNAMIC_UNPREDICTABLE_BIT
#from modules.path_planning.ARA import ARA
#from modules.path_planning.RRT_STAR_FND import RRT_STAR_FND
#from modules.path_planning.PATHADAPTIVEA import PathAdaptiveA
#from modules.path_planning.QUADRANTRRT import *

import time
import random
import math
import os
import copy
from os.path import isfile,join
#Global variables to store the mouse position coordinates from the image
startX, startY, endX, endY = 0,0,0,0
selectStart = True
selectEnd = True
img = None
im_Size = 600
img1 = None
prediction_matrix = None
path = None
path_runner = None
cv2.namedWindow('image')
cv2.namedWindow('dimage')
img_copy = None

def select_point(event,x,y,flags,param):
    """
    Grabs points selected from the cv2 image shown on screen
    """
    global startX,startY,endX,endY,img,selectStart, img1, selectEnd, prediction_matrix

        

    if event == cv2.EVENT_LBUTTONDBLCLK: # captures left button double-click
        ix,iy = x,y

        #Check to see if the user selected a point inside a road
        if not np.array_equal(img1[iy][ix],[255,255,255]):
            print("Please select a valid point inside the road!")
            return
            
        #Set the start coordinate if selectStart is still true
        if selectStart:
            img1 = cv2.circle(img1,(ix,iy),radius=5,color=(255,0,0),thickness=-1)
            img = cv2.circle(img,(ix,iy),radius=5,color=(255,0,0),thickness=-1)

            selectStart = False
            startX,startY = ix,iy
            cv2.imshow('image',img1)
            cv2.waitKey(1)
        #Set the endPoint and begin path planning computation
        elif selectEnd:
            selectEnd = False
            img1 = cv2.circle(img1,(ix,iy),radius=5,color=(0,0,255),thickness=-1)
            img = cv2.circle(img,(ix,iy),radius=5,color=(255,0,0),thickness=-1)

            endX,endY = ix,iy
            cv2.imshow('image',img1)
            cv2.imshow('dimage', img)
            cv2.waitKey(1)

            #Start the path planning algoirthm


            runPathPlanning()

            #Display the results to the user
            #Resize the image if needed
            #img = cv2.resize(img,(1024,1024),interpolation=cv2.INTER_AREA)
            cv2.imshow('image',img1)
            cv2.imshow('dimage', img)
         #   selectStart = True


def grabRandomImage():
    #Returns an image path to use for the path processing

    #imgFolder = "C:/Users/charles/Downloads/sample_predictions3/predictions"
    print("End all file paths with a /")
    imgFolder = input("Path to Image Folder: ")
   # groundTruthFolder = "C:/Users/charles/Downloads/sample_predictions3/ground_truth"
    groundTruthFolder = input("Path to ground Truth folder: ")
   # predictionFolder = input("Path to prediction folder: ")
    predictionMatrixFolder = input("Path to prediction matrix folder: ")
   # predictionFolder = "C:/Users/charles/Downloads/sample_predictions3/prediction_matrix"

    imgs = [f for f in os.listdir(imgFolder) if isfile(join(imgFolder,f))]
    img_getter = random.randint(0,len(imgs)-1)
    print(imgs[img_getter])
   # with open('C:/Users/charles/Downloads/sample_predictions3/prediction_matrix/'+imgs[img_getter][0:-3]+'pkl', 'rb') as f:
    with open(join(predictionMatrixFolder+imgs[img_getter][0:-3]+'pkl'), 'rb') as f:
        data = pickle.load(f)
    return join(imgFolder, imgs[img_getter]), join(groundTruthFolder, imgs[img_getter]), data, imgs[img_getter]
   # return join(imgFolder, 'sample_pred_1.png'), join(groundTruthFolder, 'sample_pred_1.png'), data, 'sample_pred_1.png'

def runPathPlanning(algoName="ARA"):
    start = time.time()
    startPos = (startX,startY)
    endPos = (endX,endY)
    global path
    global path_runner
    global img, img1
    global prediction_matrix

    if algoName == "astar":

        #Run the pathfinding algorithm
        path_runner = AStar(startPos,endPos,copy.deepcopy(img),"image", prediction_matrix)
        path = path_runner.run()
        
    elif algoName == "ANA":
        path_runner = ANA(startPos, endPos, img, copy.deepcopy(img), prediction_matrix)
        path = path_runner.run()
    elif algoName == "MODIFIED_ADA":
        path_runner = MODIFIED_ADA(startPos, endPos, img, prediction_matrix, img1)
        path = path_runner.Traversal()
    elif algoName == "PAA*":
        path_runner = PathAdaptiveA(startPos, endPos, img, img1, prediction_matrix )
        path = path_runner.main()
    elif algoName == "Theta":
        path_runner = Theta(startPos,endPos, copy.deepcopy(img), "image", prediction_matrix)
        pr = cProfile.Profile()
        pr.enable()
        path = path_runner.run()
        pr.disable()
        ps = pstats.Stats(pr, stream=sys.stdout)
        ps.print_stats()
        
    elif algoName == "BIT":
        path_runner = BIT(startPos, endPos, 5000, copy.deepcopy(img))
        path = path_runner.run()
    elif algoName == "bfs":
        path_runner = BFS(startPos,endPos,copy.deepcopy(img),"image")
        path = path_runner.run()
        
    elif algoName == "ARA":
        path_runner = ARA(startPos, endPos, img, prediction_matrix, 0.75, img1)
        path = path_runner.static_path_main()
        
    elif algoName == "RRTFND":
        path_runner = RRT_STAR_FND(startPos, endPos, 30, 25, img1, 20000, img1, 25, 10000)
        path = path_runner.RRT_STAR_FN_BUILD_PATH()

    elif algoName == "NEWBIT":
        path_runner = DYNAMIC_UNPREDICTABLE_BIT(startPos, endPos, 1000, copy.deepcopy(img), prediction_matrix, copy.deepcopy(img1))
        path = path_runner.run()
    elif algoName == "greedyBestFirst":
        path_runner = greedyBestFirst(startPos,endPos,copy.deepcopy(img),"image")
        path = path_runner.run()
    elif algoName == "bidirectDijkstras":
        path_runner = BiDirectionalDijkstra(startPos,endPos,copy.deepcopy(img),"image")
        path = path_runner.run()
    elif algoName == "RRT":
        path_runner = RRTtree(startPos, endPos, 4, 300, copy.deepcopy(img), "image")
        path = path_runner.run()
        
    elif algoName == "QUAD":
        path_runner = QUADRANTRRT(startPos, endPos, img, prediction_matrix, 19, 250)
        path = path_runner.region_run()
    
    elif algoName == "RRT_Star":
        path_runner = RRT_Star(startPos, endPos, 5, 10000, 50, copy.deepcopy(img1))
        path = path_runner.run()

    
    elif algoName == "INFORMED RRT*":
        path_runner = Informed_RRT_star(startPos, endPos, 8000, 30, 50, 150, copy.deepcopy(img), prediction_matrix, copy.deepcopy(img1))
        path = path_runner.run()
        print(len(path_runner.tree.vertices_and_edges))
    
    elif algoName == "RRA*":
        path_runner = RRA(startPos, endPos, img, img1)
        path = path_runner.ComputePath()
        print(path)

    elif algoName == "D* LITE":
        path_runner = DLITESEARCH(startPos, endPos, img, img1)
        path = path_runner.DLITERUN()

        
    else:
        print("No valid algorithm found for the algorithm name given...exiting")
        exit()
    #else algoName == "LPA":
    #Future path finding algorithms go here..each should return a list of coordinates
    
    end = time.time()
    duration = end - start
    print("Time took: ",duration)

    #cv2.line(img, start point, end point, color=(0, 255, 0), 3)
    #Plot the path found onto the image
    solution_quality = 0.0
    map_path = []
   # print(path_runner.current_location)
    first_point = path[0]
    second_point = None

    for pos in path:
        map_path.append(pos)
  #      cv2.rectangle(img,pos,(pos[0]+2,pos[1]+2),color=(0,0,255),thickness=-1)
        if pos != first_point:
            second_point = pos
            cv2.line(img1, first_point, second_point, color=(0, 255, 0), thickness=5)
            first_point = pos
        if pos != path[0]:
            solution_quality += math.hypot((map_path[-1][0] - map_path[-2][0]),(map_path[-1][1] - map_path[-2][1]))

    img1 = cv2.circle(img1, (startX, startY), radius=5, color=(255, 0, 0), thickness=-1)
    img1 = cv2.circle(img1, (endX, endY), radius=5, color=(0, 0, 255), thickness=-1)


    print('Quality/Cost of solution: ', solution_quality)
    

def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    global img 
    global img1
    global prediction_matrix
    global im_Size
    imgPath, imgPathgroundTruth, prediction_matrix, filename = grabRandomImage()

    #Load the image
    img = cv2.resize(cv2.imread(imgPath), (im_Size, im_Size))
    img1 = cv2.resize(cv2.imread(imgPathgroundTruth), (im_Size, im_Size))
    imcopy = copy.deepcopy(img)
    prediction_matrix = cv2.resize(prediction_matrix, (im_Size,im_Size))

    #Resize the image if desired (Speeds up computation)
    #img = cv2.resize(img,(512,512),interpolation=cv2.INTER_AREA)
    #Convert the image to GrayScale
    #img = cv2.cvtColor(cv2.imread('path1.png'),cv2.COLOR_BGR2GRAY)

    #Initialize the image window
    cv2.setMouseCallback('image',select_point)

   # cv2.setMouseCallback('image', lambda *args : None)
    #Display image and wait until the user selects two valid points on the image 
    cv2.imshow('image',img1)
    cv2.imshow('dimage', img)
    cv2.imshow('imcopy', imcopy)

    k = cv2.waitKey(0) 
    direct = r'C:\Users\charles\Desktop\ARA'+filename
        
    cv2.imwrite(direct, img1)

    cv2.destroyAllWindows()



if __name__ == "__main__":
    main()

End all file paths with a /
Path to Image Folder: C:/Users/charles/Downloads/sample_predictions3/predictions/
Path to ground Truth folder: C:/Users/charles/Downloads/sample_predictions3/ground_truth
Path to prediction matrix folder: C:/Users/charles/Downloads/sample_predictions3/prediction_matrix/
sample_pred_7.png
Please select a valid point inside the road!
Please select a valid point inside the road!
Please select a valid point inside the road!
Please select a valid point inside the road!
Please select a valid point inside the road!
Please select a valid point inside the road!
Beginning ARA* search from (41, 110) to (157, 401)
[[0.99970466]
 [0.01194626]
 [0.4605714 ]
 [0.7581243 ]
 [0.22256237]]
Initializing tree:
ARA* initial path has an accuracy of:  11.643835616438356 %
Time took:  4.366173505783081
Quality/Cost of solution:  376.3279938488588
